<a href="https://colab.research.google.com/github/AniketGaudgaul/AI-ML-DL_Projects/blob/main/SC3_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download aseemdandgaval/23-pet-breeds-image-classification
!unzip /content/23-pet-breeds-image-classification.zip

In [ ]:
import os
import numpy as np
from PIL import Image

# Set the input path and target size for the images
input_folder = '/content/Pet_Breeds'

# Loop through all the folders in the input folder
for i, folder_name in enumerate(os.listdir(input_folder)):
    folder_path = os.path.join(input_folder, folder_name)

    # Loop through all the files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Open the image and resize it
        with Image.open(file_path) as image:
            if image.mode not in ('RGB', 'RGBA'):
                image = image.convert('RGB')
            if image.mode == 'RGBA':
                image = image.convert('RGB')



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

input_folder = '/content/Pet_Breeds'
target_size = (224, 224)
batch_size = 50

# Create an instance of the ImageDataGenerator class for training and validation data with data augmentation
data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10, # randomly rotate images up to 20 degrees
    width_shift_range=0.1, # randomly shift images horizontally by up to 10% of the image width
    height_shift_range=0.1, # randomly shift images vertically by up to 10% of the image height
    zoom_range=0.1, # randomly zoom images by up to 10%
    horizontal_flip=True # randomly flip images horizontally
)

# Load the training data with data augmentation
train_data = data_generator.flow_from_directory(
    input_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load the validation data without data augmentation
val_data = data_generator.flow_from_directory(
    input_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


# Get the class indices from the data generator
class_indices = train_data.class_indices

# Reverse the class indices dictionary to get a dictionary of breed names to indices
breed_to_index = dict((v,k) for k,v in class_indices.items())

# Use the breed_to_index dictionary to get a list of the breed names for each image in the training and validation sets
train_breeds = [breed_to_index[np.argmax(train_data[i][1])] for i in range(len(train_data))]
val_breeds = [breed_to_index[np.argmax(val_data[i][1])] for i in range(len(val_data))]

# Convert the list of breeds to numpy arrays
train_breeds = np.array(train_breeds)
val_breeds = np.array(val_breeds)

# Print the shape of the arrays
print("Training breeds shape:", train_breeds.shape)
print("Validation breeds shape:", val_breeds.shape)

Found 3105 images belonging to 23 classes.
Found 776 images belonging to 23 classes.
Training breeds shape: (63,)
Validation breeds shape: (16,)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Define the input shape for the MobileNet model
input_shape = (224, 224, 3)

# Load the MobileNet model, pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers of the model so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(23, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_data, epochs=3, validation_data=val_data)


Epoch 1/3
63/63 [==============================] - 166s 2s/step - loss: 0.9329 - accuracy: 0.7314 - val_loss: 0.6829 - val_accuracy: 0.8028
Epoch 2/3
63/63 [==============================] - 147s 2s/step - loss: 0.4008 - accuracy: 0.8699 - val_loss: 0.8300 - val_accuracy: 0.7938
Epoch 3/3
63/63 [==============================] - 135s 2s/step - loss: 0.3085 - accuracy: 0.9011 - val_loss: 0.4803 - val_accuracy: 0.8505


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image
img_path = '/content/mainecoon.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image
img_array /= 255.

# Make a prediction
prediction = model.predict(img_array)

# Get the predicted breed
predicted_breed = breed_to_index[np.argmax(prediction)]

# Print the predicted breed
print("Predicted breed:", predicted_breed)


1/1 [==============================] - 0s 27ms/step
Predicted breed: maine coon


In [ ]:
# Save the trained model
model.save('my_pet_breeds_MobileNet.h5')


In [ ]:
from google.colab import files
files.download('/content/my_pet_breeds_MobileNet.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>